# INTEREST RATE DOESN'T WORK STILL

set r = 0 for all cases

In [1]:
##### sim
import random
import matplotlib.pyplot as plt

def getConsumption(assets: float, expectedincomepath: list, periodevaluated: int, interestrate: float) -> float:
    # print(assets)
    restoflife = expectedincomepath[periodevaluated-1: ]
    incomesum = sum(restoflife)
    lifetimeincome = assets + incomesum
    c1 = lifetimeincome / len(restoflife)
    interestsum = 0
    for y in restoflife:
        interest = interestrate * (y - c1)
        interestsum += interest
    c1 += (interestsum / len(restoflife))
    return round(c1, 4)

class Smoothing():
    def __init__(self, INCOME_PER_PERIOD, PERIODS, ASSETS, R) -> None:
        self.PERIODS = PERIODS
        self.R = R

        randis = random.sample(range(75, 125, 5), k=3)
        randpsexpected = random.sample(range(2, 7), k=2)
        randpsunexpected = random.sample(range(7, 11), k=1)

        expected = {randpsexpected[0]: randis[0], randpsexpected[1]: randis[1]}
        unexpected = {randpsunexpected[0]: randis[2]} 
        unexpectedperiods = list(unexpected.keys())
        
        initialincomepath = []
        for i in range(PERIODS):
            initialincomepath.append(INCOME_PER_PERIOD)
        for i in expected:
            initialincomepath[i-1] = expected[i]

        expectedincomepath = initialincomepath.copy()

        period = 1
        assetpath = [ASSETS]
        assets = ASSETS
        consumptionpath = []
        consumption = getConsumption(assets, expectedincomepath, period, R)
        while period <= PERIODS:
            assets = assets * (1+R)
            if period in unexpectedperiods:
                expectedincomepath[period-1] = unexpected[period]
            consumption = getConsumption(assets, expectedincomepath, period, R)
            assetpath.append(expectedincomepath[period-1] - consumption)
            consumptionpath.append(consumption)
            assets = sum(assetpath)
            period += 1

        self.initialincomepath = initialincomepath
        self.expectedincomepath = expectedincomepath
        self.consumptionpath = consumptionpath
        self.assetpath = assetpath
        self.expected = expected
        self.unexpected = unexpected


    def PrintStuff(self) -> None:
        print(f'Rate of Interest: {self.R * 100}%')
        print('Initial income path: ', self.initialincomepath)
        print('Expected Shocks (period: shock income): ', self.expected)
        print('Unexpected Shocks (period: shock income): ', self.unexpected)
        print('Actual income path: ', self.expectedincomepath)
        print('Consumption Path: ', self.consumptionpath)
        print('Asset path: ', self.assetpath)
        print('Sum of consumption: ', round(sum(self.consumptionpath)))
        print('Sum of assets: ', round(sum(self.assetpath)))

    def PlotStuff(self):
        index = 0
        while index < (self.PERIODS):
            plt.plot([index, index+1], [self.expectedincomepath[index], self.expectedincomepath[index]], c='b')
            plt.plot([index, index+1], [self.initialincomepath[index], self.initialincomepath[index]], c='g')
            plt.plot([index, index+1], [self.consumptionpath[index], self.consumptionpath[index]], c='r')
            
            if index != (self.PERIODS - 1):
                plt.plot([index+1, index+1], [self.expectedincomepath[index], self.expectedincomepath[index+1]], c='gray')
                plt.plot([index+1, index+1], [self.initialincomepath[index], self.initialincomepath[index+1]], c='gray')
                plt.plot([index+1, index+1], [self.consumptionpath[index], self.consumptionpath[index+1]], c='gray')
            index+=1

        plt.show()

In [2]:
import ipywidgets as widgets

In [3]:
out = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 0 0 0'})
out

Output(layout=Layout(border='2px solid gray', margin='0 0 0 0', padding='5px 5px 5px 5px'))

In [4]:
inc = widgets.IntSlider(description='Income Per Period', value=100, min=50, max=150)
ass = widgets.IntSlider(description='Initial Assets', value=0, min=-50, max=50)
r = widgets.IntSlider(description='Interest Rate', value=0, min=0, max=25)
seed = widgets.IntText(value=random.randint(0, 10000), description='random seed')
sub = widgets.Button(description='Submit')

random.seed(seed.value)

with out:
    out.clear_output(wait=True)
    display(inc, ass, r, seed, sub)

def on_sub(b):
    out.clear_output(wait=True)
    random.seed(seed.value)
    sim = Smoothing(inc.value, 10, ass.value, 0.01 * r.value)
    with out:
        display(inc, ass, r, seed, sub)
        sim.PrintStuff()
        sim.PlotStuff()

sub.on_click(on_sub)